In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd '/content/drive/MyDrive/Waterloo/_MSCI_641/project'

/content/drive/MyDrive/Waterloo/_MSCI_641/project


In [3]:
!pip install virtualenv
!pip install rouge_score
!pip install transformers
!pip install wandb
!pip install datasets
!pip install evaluate
!pip install accelerate
# !pip install gradio --upgrade
# !pip install --upgrade torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.9/468.9 kB 33.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=eb78f3da8625b1f6d74dbf2963cbbc769abd3c8d3adb41807163d2a4e4f954af
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━

In [4]:
import json, os
import numpy as np
from utils import *
import pandas as pd
import evaluate
import torch, wandb
from tqdm.auto import tqdm
from datasets import Dataset
from torch.utils.data import DataLoader
from transformers import default_data_collator
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, AutoModelForSequenceClassification, AdamW, RobertaTokenizer
from collections import defaultdict
from transformers import Trainer, TrainingArguments
from accelerate import Accelerator
from transformers import get_scheduler
# import gradio as gr
from sklearn.metrics import f1_score, accuracy_score
import gc
os.environ["TOKENIZERS_PARALLELISM"] = "false"

tqdm.pandas()

In [5]:
datasets = dict()
dataset_names = ['train','val','test']

for dataset_name in dataset_names:
    datasets[dataset_name] = pd.read_json(path_or_buf=dataset_name + '.jsonl', lines=True)
    if 'id' not in datasets[dataset_name].columns:
        datasets[dataset_name]['id'] = datasets[dataset_name].index

features = ['postText', 'postPlatform', 'targetParagraphs', 'targetTitle',
       'targetDescription', 'targetKeywords', 'targetMedia', 'targetUrl']
features = ['postText', 'postPlatform', 'targetParagraphs', 'targetTitle',
       'targetDescription', 'targetKeywords', 'targetMedia', 'targetUrl']
features_text = ['postText','targetParagraphs', 'targetTitle', 'targetDescription',
                'targetKeywords']
features_url = ['targetUrl', 'targetMedia' ]

target1 = 'tags'
target2 = 'spoiler'

In [8]:


label_encoder = {"phrase": 0,"passage": 1,"multi":2}
label_decoder = {0:"phrase",1:"passage", 2:"multi"}

for dataset_name in dataset_names:

    print(dataset_name)

    temp_data = datasets[dataset_name]

    if dataset_name != 'test':
        temp_data = temp_data[temp_data['spoilerPositions'].apply(lambda x: len(x[0])) > 1]
        temp_data['tokPos'] = temp_data.apply(findPosTags,1)

        temp_data['tags'] = temp_data['tags'].apply(lambda x: x[0])
        temp_data['label'] = temp_data['tags'].map(label_encoder)
        temp_data['tokenPosition'] = temp_data.apply(findPosTags,1)
        temp_data['longest_spoiler_index'] = temp_data['spoiler'].apply(return_longest_spoiler_index)
        temp_data['longest_spoiler'] = temp_data.apply(lambda x: x['spoiler'][x['longest_spoiler_index']],1)
        temp_data['longest_tokPos'] = temp_data.apply(lambda x: x['tokPos'][x['longest_spoiler_index']],1)

    temp_data['postText'] = temp_data['postText'].apply(lambda x: x[0])

    for col in ['targetParagraphs', 'targetMedia']:

        print(col)

        temp_data[col + '_listlen'] = temp_data[col].apply(lambda x: len(x) if x is not None else 0)
        temp_data[col + '_concat'] = temp_data[col].apply(lambda x: ' '.join(x) if x is not None else '')

    for col in ['postText', 'targetTitle', 'targetDescription', 'targetKeywords', 'targetParagraphs_concat']:

        print(col)
        temp_data[col + '_len'] = temp_data[col].apply(lambda x: len(x.split()) if x is not None else 0)

    temp_data['title_para'] = temp_data['targetTitle'] + " - " + temp_data['targetParagraphs_concat']

    datasets[dataset_name] = temp_data

train
targetParagraphs
targetMedia
postText
targetTitle
targetDescription
targetKeywords
targetParagraphs_concat


<ipython-input-8-0d1bb7c1b72f>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_data['tokPos'] = temp_data.apply(findPosTags,1)
<ipython-input-8-0d1bb7c1b72f>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_data['tags'] = temp_data['tags'].apply(lambda x: x[0])
<ipython-input-8-0d1bb7c1b72f>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.

val
targetParagraphs
targetMedia
postText
targetTitle
targetDescription
targetKeywords
targetParagraphs_concat
test
targetParagraphs
targetMedia
postText
targetTitle
targetDescription
targetKeywords
targetParagraphs_concat


<ipython-input-8-0d1bb7c1b72f>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_data[col + '_len'] = temp_data[col].apply(lambda x: len(x.split()) if x is not None else 0)
<ipython-input-8-0d1bb7c1b72f>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_data['title_para'] = temp_data['targetTitle'] + " - " + temp_data['targetParagraphs_concat']


In [10]:
datasets['test'].iloc[8]

,8
postId,53e76y
postText,The First Person In The Royal Family Has Just Come Out As Gay
postPlatform,reddit
targetParagraphs,"[The Queen’s cousin has become the first member of the British Royal Family to come out as gay., Lord Ivar Mountbatten has revealed that he is now dating a man after a lifelong struggle with his sexuality., Prince Harry previously joked that he might try a relationship with a man if it doesn’t work out with women., Mountbatten is the Queen’s cousin, the great great great grandson of Queen Victoria and the great-nephew of Earl Mountbatten of Burma., The 53-year-old came out in an interview with his boyfriend James Coyle, 54. The couple met on a skiing holiday in March, and the relationship has been a key landmark for them both., He is the first member of the extended royal family to come out as gay, and whilst he is happier now he’s still finding his feet in his new relationship., ""I am a lot happier now, though I am still not 100% comfortable with being gay,"" he told The Mail on Sunday., ""Being a Mountbatten was never the problem, it was the generation into which I was born. When I was growing up, it was known as ‘the love that dare not speak its name’, but what’s amazing now is how far we have all come in terms of acceptance."", ""Coming out is such a funny phrase but it’s what I suppose I did in a rather roundabout way, emerging to a place I’m happy to be. I have struggled with my sexuality and in some ways I still do; it has been a real journey to reach this point."", ""Simply talking about it in public is a huge step for me. Up to this point, I have had a heterosexual lifestyle, so living with a man is really new. One step at a time."", The father of three divorced from his wife Penelope Thompson after 17 years of marriage in 2011., Whilst having a boyfriend has provided the Lord with some learning curves, it’s also been a blessing in disguise. His daughters like having another man in the house, rather than a step-mother. And his ex, Penelope Thomson, goes to yoga with his new boyfriend., Coyle said: ""I went to a village yoga class with Penny and people thought I was her new boyfriend. She had to tell them: no, he’s Ivar’s, not mine."", Whilst the couple have a strict no public affection rule, they are now out to all of their friends and family., ""In different ways, we both struggled. Now everyone in our family knows and could not be more supportive."" said Mountbatten. ""Neither of us wanted to have relationships which were transitory; the stereotypical view of gay relationships is that they are too fleeting, too frequent."", ""Finding James means I will not have to lie to anyone or grow old on my own."", Last year, Prince William visited a school to participate in a lesson to tackle homophophic bullying., The Queen is believed to have supported same-sex marriage. When a newspaper claimed that she opposed gay people marrying, Buckingham Palace sent journalists PinkNews articles claiming that she supports equality.]"
targetTitle,The first person in the royal family has just come out as gay
targetDescription,The Queen’s cousin has become the first member of the royal family to come out as gay.
targetKeywords,None
targetMedia,"[http://cdn.pinknews.co.uk/images/2016/08/truvada-93x70.jpg, http://cdn.pinknews.co.uk/images/2015/07/close.png, http://cdn.pinknews.co.uk/wp-content/themes/pn-responsive-full/images/pn_mobile_logo.png, http://0.gravatar.com/avatar/fc7bd9632bf87b2e3280355bdac262e4?s=25&d=mm&r=g, http://cdn.pinknews.co.uk/images/2016/09/queens-cousin-1_640x345_acf_cropped.jpg, http://cdn.pinknews.co.uk/images/2016/07/harrydi-93x70.jpg, http://cdn.pinknews.co.uk/wp-content/themes/pn-responsive-full/images/pn_new_logo.png, http://pixel.quantserve.com/pixel/p-b9cuJqjzzENZA.gif, http://cdn.pinknews.co.uk/images/2016/07/Prince-Harry-HIV2-93x70.jpg, http://cdn.pinknews.co.uk/images/2016/06/150903-queen-elizabeth-jpo-712a_80834894bceed6fc1635f6a6ac96ab46.nbcnews-ux-2880-1000.jpg, http://cdn.pinknew

In [11]:
datasets['train']['tags']

,tags
0,passage
1,phrase
2,phrase
3,multi
4,phrase
5,passage
6,passage
7,passage
8,passage
9,multi


In [13]:
datasets['train'][['spoiler','tokPos','longest_spoiler', 'spoilerPositions','longest_tokPos']]

,spoiler,tokPos,longest_spoiler,spoilerPositions,longest_tokPos
0,[how about that morning we go throw?],"[[772, 807]]",how about that morning we go throw?,"[[[3, 151], [3, 186]]]","[772, 807]"
1,[2070],"[[67, 71]]",2070,"[[[0, 0], [0, 4]]]","[67, 71]"
2,[intellectual stimulation],"[[349, 373]]",intellectual stimulation,"[[[1, 186], [1, 210]]]","[349, 373]"
3,[Purpose connects us to something bigger and i...,"[[2360, 2436], [3808, 3837], [5148, 5214], [63...",Purpose connects us to something bigger and in...,"[[[11, 25], [11, 101]], [[17, 56], [17, 85]], ...","[2360, 2436]"
4,[in a rice cooker],"[[769, 785]]",in a rice cooker,"[[[5, 60], [5, 76]]]","[769, 785]"
...,...,...,...,...,...
3195,[it hasn’t necessarily taken the wind out of Y...,"[[1834, 1902]]",it hasn’t necessarily taken the wind out of Yo...,"[[[7, 50], [7, 118]]]","[1834, 1902]"
3196,"[It read, ""Thanks for keeping us safe.""]","[[416, 454]]","It read, ""Thanks for keeping us safe.""","[[[0, 317], [0, 355]]]","[416, 454]"
3197,[1. Anti-wrinkle creams will erase the fine li...,"[[873, 946], [1441, 1550], [2266, 2329], [3039...",2. Dark circles will appear beneath your eyes ...,"[[[6, 0], [6, 73]], [[10, 0], [10, 109]], [[14...","[1441, 1550]"
3198,[@beyoncefan666],"[[495, 509]]",@beyoncefan666,"[[[3, 55], [3, 69]]]","[495, 509]"


In [14]:
datasets['train'].columns

Index(['uuid', 'postId', 'postText', 'postPlatform', 'targetParagraphs',
       'targetTitle', 'targetDescription', 'targetKeywords', 'targetMedia',
       'targetUrl', 'provenance', 'spoiler', 'spoilerPositions', 'tags', 'id',
       'tokPos', 'label', 'tokenPosition', 'longest_spoiler_index',
       'longest_spoiler', 'longest_tokPos', 'targetParagraphs_listlen',
       'targetParagraphs_concat', 'targetMedia_listlen', 'targetMedia_concat',
       'postText_len', 'targetTitle_len', 'targetDescription_len',
       'targetKeywords_len', 'targetParagraphs_concat_len', 'title_para'],
      dtype='object')

In [15]:
bleu = evaluate.load("bleu")
squad_metric = evaluate.load("squad")

In [16]:
meteor = evaluate.load("meteor")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [17]:
rouge = evaluate.load("rouge")

In [18]:
train_df = convert2squadFormat_qa(datasets['train'], 'train')
val_df = convert2squadFormat_qa(datasets['val'], 'val')
test_df = convert2squadFormat_qa(datasets['test'], 'test')

train_data = Dataset.from_pandas(train_df.reset_index(drop=True), split="train")
val_data = Dataset.from_pandas(val_df.reset_index(drop=True), split="val")
test_data = Dataset.from_pandas(test_df.reset_index(drop=True), split="test")

<ipython-input-12-4d217fc47e54>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fin["asnwers"] = df_fin.apply(lambda x: {'text':[x['longest_spoiler']], "answer_start":[x['longest_tokPos'][0]]},1)
<ipython-input-12-4d217fc47e54>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fin["asnwers"] = df_fin.apply(lambda x: {'text':[x['longest_spoiler']], "answer_start":[x['longest_tokPos'][0]]},1)
<ipython-input-12-4d217fc47e54>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

In [19]:
train_df

,id,labels,title,question,context,answers
0,0,1,"Wes Welker Wanted Dinner With Tom Brady, But P...","Wes Welker Wanted Dinner With Tom Brady, But P...","Wes Welker Wanted Dinner With Tom Brady, But P...",{'text': ['how about that morning we go throw?...
1,1,0,Hole In Ozone Layer Expected To Make Full Reco...,NASA sets date for full recovery of ozone hole,Hole In Ozone Layer Expected To Make Full Reco...,"{'text': ['2070'], 'answer_start': [67]}"
2,2,0,Intellectual Stimulation Trumps Money For Empl...,This is what makes employees happy -- and it's...,Intellectual Stimulation Trumps Money For Empl...,"{'text': ['intellectual stimulation'], 'answer..."
3,3,2,"‘Follow your passion’ is wrong, here are 7 hab...",Passion is overrated — 7 work habits you need ...,"‘Follow your passion’ is wrong, here are 7 hab...",{'text': ['Purpose connects us to something bi...
4,4,0,Revealed: The perfect way to cook rice so that...,The perfect way to cook rice so that it's perf...,Revealed: The perfect way to cook rice so that...,"{'text': ['in a rice cooker'], 'answer_start':..."
...,...,...,...,...,...,...
3195,3195,1,"Facebook Video Surging, But YouTube Still Offe...",Has Facebook's video explosion completely shak...,"Facebook Video Surging, But YouTube Still Offe...",{'text': ['it hasn’t necessarily taken the win...
3196,3196,1,Cop is eating at Chili's when teen hands him f...,Cop Is Eating At A Chili's When Teen Hands Him...,Cop is eating at Chili's when teen hands him f...,"{'text': ['It read, ""Thanks for keeping us saf..."
3197,3197,2,5 popular myths about visible signs of aging t...,5 popular myths about visible signs of aging t...,5 popular myths about visible signs of aging t...,{'text': ['2. Dark circles will appear beneath...
3198,3198,0,"WTF, It Looks Like This Twitter Account ""Predi...",You need to see this Twitter account that pred...,"WTF, It Looks Like This Twitter Account ""Predi...","{'text': ['@beyoncefan666'], 'answer_start': [..."


In [34]:
model_name = "csarron/bert-base-uncased-squad-v1"

model_name = "deepset/minilm-uncased-squad2"

model_name = "facebook/bart-base"

model_name = "sjrhuschlee/bart-base-squad2"

model_name = "FacebookAI/roberta-base"

model_name = "deepset/roberta-base-squad2"


# Phrase Generation
config = dict(
max_length = 512,
stride = 128,
n_best = 10,
max_answer_length = 30,
batch_size = 16,
epochs = 10,
learning_rate = 1e-5,
model_name = model_name,
)


max_length = config["max_length"]
stride = config["stride"]
n_best = config["n_best"]
max_answer_length = config["max_answer_length"]


tokenizer = AutoTokenizer.from_pretrained(config["model_name"])
model = AutoModelForQuestionAnswering.from_pretrained(config["model_name"])

In [36]:
train_dataset = train_data.map(
    preprocess_training_examples_qa,
    batched=True,
    remove_columns=train_data.column_names)

Map:   0%|          | 0/3199 [00:00<?, ? examples/s]

In [37]:
validation_dataset = val_data.map(
    preprocess_training_examples_qa,
    batched=True,
    remove_columns=val_data.column_names,
)

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

In [38]:
test_dataset = test_data.map(
    preprocess_test_examples_qa,
    batched=True,
    remove_columns=test_data.column_names,
)

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

len(inputs["input_ids"]) 849


In [39]:
train_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'offset_mapping', 'example_id', 'start_positions', 'end_positions'],
    num_rows: 6578
})

In [40]:
test_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'offset_mapping', 'example_id'],
    num_rows: 849
})

In [41]:
validation_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'offset_mapping', 'example_id', 'start_positions', 'end_positions'],
    num_rows: 863
})

In [42]:
len(train_dataset), len(validation_dataset), len(test_dataset)

(6578, 863, 849)

In [43]:
train_set = train_dataset.remove_columns(["example_id", "offset_mapping"])
validation_set = validation_dataset.remove_columns(["example_id", "offset_mapping"])
test_set = test_dataset.remove_columns(["example_id", "offset_mapping"])
train_set.set_format("torch")
validation_set.set_format("torch")
test_set.set_format("torch")

train_dataloader = DataLoader(
    train_set,
    collate_fn=default_data_collator,
    batch_size=config["batch_size"],
)

eval_dataloader = DataLoader(
    validation_set, collate_fn=default_data_collator, batch_size=32
)
test_dataloader = DataLoader(
    test_set, collate_fn=default_data_collator, batch_size=32
)

optimizer = AdamW(model.parameters(), lr=config["learning_rate"])
accelerator = Accelerator(mixed_precision ='fp16')
model, optimizer, train_dataloader, eval_dataloader, test_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader, test_dataloader
)

num_train_epochs = config["epochs"]
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [44]:
stats = pd.DataFrame()
batch = None
for epoch in range(num_train_epochs):
    print("Starting epoch: ", epoch)
    # Training
    model.train()
    train_cls_loss = 0
    train_qa_loss = 0
    start_logits_t = []
    end_logits_t = []
    class_logits_t = []
    for batch in tqdm(train_dataloader):
        outputs = model(**batch)
        qa_loss = outputs['loss']
        accelerator.backward(qa_loss)
        train_qa_loss+=qa_loss.item()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        start_logits_t.append(accelerator.gather(outputs['start_logits']).detach().cpu().numpy())
        end_logits_t.append(accelerator.gather(outputs['end_logits']).detach().cpu().numpy())

    start_logits = np.concatenate(start_logits_t)
    end_logits = np.concatenate(end_logits_t)
    start_logits = start_logits[: len(train_dataset)]
    end_logits = end_logits[: len(train_dataset)]

    metrics,theoretical_texts,predicted_texts = compute_metrics_qa(
        start_logits, end_logits, train_dataset, train_data
    )
    del start_logits, end_logits, start_logits_t, end_logits_t, outputs, batch
    torch.cuda.empty_cache()
    gc.collect()

    train_metrics = {'train_' + k:v for k,v in metrics.items()}

    # Evaluation
    model.eval()
    start_logits_t = []
    end_logits_t = []
    class_logits_t = []
    accelerator.print("Evaluation!")

    val_qa_loss = 0
    for batch in tqdm(eval_dataloader):
        with torch.no_grad():
            outputs = model(**batch)
            val_qa_loss += outputs['loss'].item()

        start_logits_t.append(accelerator.gather(outputs['start_logits']).cpu().numpy())
        end_logits_t.append(accelerator.gather(outputs['end_logits']).cpu().numpy())

    start_logits = np.concatenate(start_logits_t)
    end_logits = np.concatenate(end_logits_t)
    start_logits = start_logits[: len(validation_dataset)]
    end_logits = end_logits[: len(validation_dataset)]

    metrics,theoretical_texts,predicted_texts = compute_metrics_qa(
        start_logits, end_logits, validation_dataset, val_data
    )

    val_metrics = {'val_' + k:v for k,v in metrics.items()}

    metrics = train_metrics
    for k,v in val_metrics.items():
      metrics[k] = v

    metrics['epoch'] = epoch
    train_qa_loss = train_qa_loss/len(train_dataloader)
    val_qa_loss = val_qa_loss/len(eval_dataloader)
    metrics['train_qa_loss'] = train_qa_loss
    metrics['val_qa_loss'] = val_qa_loss

    print("Epoch stats: ")
    print(metrics)

    df_metrics = pd.DataFrame(metrics.values()).T
    df_metrics.columns=metrics.keys()
    stats = pd.concat([stats,df_metrics])

    stats.to_csv("results_qa_only_robertasquad_new/metrics_highLR.csv")

    model.save_pretrained(f"./models_task2_/")

Starting epoch:  0


  0%|          | 0/412 [00:00<?, ?it/s]

  0%|          | 0/3199 [00:00<?, ?it/s]

Evaluation!


  0%|          | 0/27 [00:00<?, ?it/s]

...

  0%|          | 0/27 [00:00<?, ?it/s]

  0%|          | 0/400 [00:00<?, ?it/s]

Epoch stats: 
{'train_squad_exact_match': 0.0313, 'train_squad_f1': 4.9661, 'train_bleu': 0.0243, 'train_meteor': 0.0547, 'train_rouge1': 0.058, 'train_rouge2': 0.0173, 'train_rougeL': 0.0521, 'train_rougeLsum': 0.0522, 'val_squad_exact_match': 0.0, 'val_squad_f1': 2.8074, 'val_bleu': 0.0055, 'val_meteor': 0.0314, 'val_rouge1': 0.0344, 'val_rouge2': 0.0057, 'val_rougeL': 0.0307, 'val_rougeLsum': 0.0308, 'epoch': 1, 'train_qa_loss': 6.248200055464958, 'val_qa_loss': 6.23828125}
Starting epoch:  2


  0%|          | 0/412 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
model.eval()
start_logits = []
end_logits = []
accelerator.print("Evaluation!")
model.train()

for batch in eval_dataloader:
    with torch.no_grad():
        outputs = model(**batch)

    start_logits.append(accelerator.gather(outputs['start_logits']).cpu().numpy())
    end_logits.append(accelerator.gather(outputs['end_logits']).cpu().numpy())

start_logits = np.concatenate(start_logits)
end_logits = np.concatenate(end_logits)
start_logits = start_logits[: len(validation_dataset)]
end_logits = end_logits[: len(validation_dataset)]

metrics,theoretical_texts,predicted_texts = compute_metrics_qa(
    start_logits, end_logits, validation_dataset, val_data
)

In [ ]:
metrics

In [ ]:
model.eval()
start_logits = []
end_logits = []
accelerator.print("Evaluation!")

for batch in tqdm(test_dataloader):
    with torch.no_grad():
        outputs = model(**batch)

    start_logits.append(accelerator.gather(outputs['start_logits']).cpu().numpy())
    end_logits.append(accelerator.gather(outputs['end_logits']).cpu().numpy())

start_logits = np.concatenate(start_logits)
end_logits = np.concatenate(end_logits)
start_logits = start_logits[: len(test_dataset)]
end_logits = end_logits[: len(test_dataset)]

predicted_texts = compute_metrics_qa(
    start_logits, end_logits, test_dataset, test_data, predictOnly=True
)


In [ ]:
df_test_submission = pd.DataFrame(columns=['id','spoiler'])
df_test_submission['id'] = test_data['id']
df_test_submission['spoiler'] = predicted_texts
df_test_submission.to_csv('results_qa_only_robertasquad_new/submission_T2_new.csv',index=False)
df_test_submission

In [ ]:
df_test_submission[df_test_submission['spoiler'].isna()]

In [ ]:
s